In [123]:
from transformers import PreTrainedTokenizerFast, BartForConditionalGeneration
# from datasets import Dataset, load_dataset
# from torch.utils.data import DataLoader

import torch
import pandas as pd
import numpy as np
import time
# import sys
# import requests
# import json
from collections import defaultdict
import os
import pickle
import re

# from newspaper import Article
# from sklearn.metrics.pairwise import cosine_similarity
import networkx as nx
from hanspell import spell_checker

In [ ]:
df = pd.read_pickle('search/')

In [15]:
class SummaryGenerater():
    def __init__(self, model, tokenizer, file_path):
        self.model = model
        self.tokenizer = tokenizer
        self.df = pd.read_pickle(file_path).dropna().reset_index(drop=True)
        self.model.to(device)
        self.topic_nums = sorted(self.df['topic'].unique())
    
    def concat_title_context(self):
        df = self.df
        add_title_context = []
        for _, t in df.iterrows():
            context = [t['title']] + t['context'][0:2]
            add_title_context.append((' '.join(context)).strip())
        df['concat_text'] = add_title_context
        return df

    def screened_articles(self, threshold=0.3):
        df = self.concat_title_context()
        indexes = []
        for topic_n in self.topic_nums:
            if topic_n == -1:
                continue
            matrix = []
            topic_df = df[df['topic'] == topic_n]
            idx = topic_df.index
            length = len(topic_df)
            # print(f'Num of articles on topic {topic_n}: {length}')
            for c1 in topic_df['concat_text']:
                tmp = []
                for c2 in topic_df['concat_text']:
                    s1 = set(c1)
                    s2 = set(c2)
                    actual_jaccard = float(len(s1.intersection(s2)))/float(len(s1.union(s2)))
                    tmp.append(actual_jaccard)
                matrix.append(tmp)
            matrix = np.array(matrix)
            matrix = np.array(matrix > threshold)
            G = nx.from_numpy_array(matrix)
            attribute_sorted = sorted(G.degree, key=lambda x: x[1], reverse=True)
            article_idx = [idx for (idx, _) in attribute_sorted if (attribute_sorted[0][1] - int(length*0.1)) <= _]
            if len(attribute_sorted) >= 3 and len(article_idx) < 3:
                article_idx = [idx for (idx, _) in attribute_sorted[:3]]
            elif len(article_idx) > 12:
                article_idx = [idx for (idx, _) in attribute_sorted[:12]]
            # print(f'Num of articles after screening: {len(article_idx)}')
            indexes += list(idx[article_idx])
        return df.iloc[indexes]
    
    def summary(self):
        summary_dict = {}
        df = self.screened_articles()
        for topic_n in self.topic_nums:
            if topic_n == -1:
                continue
            topic_context = list(df[df['topic'] == topic_n]['concat_text'])
            s = time.time()
            input_ids = self.tokenizer.encode('.'.join(topic_context))
            summary_ids = self.model.generate(torch.tensor([input_ids[:1020]]).to(device), num_beams=3, max_length=256, eos_token_id=1)
            summary_text = self.tokenizer.decode(summary_ids.squeeze().tolist(), skip_special_tokens=True)
            summary_dict[topic_n] = summary_text
            print("================   Topic #: ", topic_n)
            print("================   Article #: ", len(topic_context))
            # print(topic_context)
            # print("================")
            # print(topic_n, summary_text)
            print(f'{(time.time() - s):0.2f} sec')
        return summary_dict

In [16]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

In [17]:
tokenizer = PreTrainedTokenizerFast.from_pretrained('digit82/kobart-summarization')
model = BartForConditionalGeneration.from_pretrained('digit82/kobart-summarization')

You passed along `num_labels=3` with an incompatible id to label map: {'0': 'NEGATIVE', '1': 'POSITIVE'}. The number of labels wil be overwritten to 2.


title, url, date, context, topic // topic, summary1

In [26]:
files = [file for file in os.listdir('search/kmeans_4') if file[-4:] == '.pkl']
for file in files:
    file_path = 'search/kmeans_4/' + file
    print(file)
    summary_model = SummaryGenerater(model, tokenizer, file_path)
    articles = summary_model.screened_articles()
    atricle_summary = summary_model.summary()
    articles.to_pickle(f'pass/kmeans_4/{file}')
    with open(f'summary/kmeans_4/{file}', 'wb') as f:
        pickle.dump(atricle_summary, f)

kmeans_4_삼성전자_20221201_20221215_crwal_news_context.pkl
================   Topic #:  0
================   Article #:  12
0.34 sec
================   Topic #:  1
================   Article #:  5
0.41 sec
================   Topic #:  2
================   Article #:  3
0.31 sec
================   Topic #:  3
================   Article #:  12
0.55 sec
kmeans_4_삼성전자_20221201_20221231_crwal_news_context.pkl
================   Topic #:  0
================   Article #:  5
0.44 sec
================   Topic #:  1
================   Article #:  4
0.31 sec
================   Topic #:  2
================   Article #:  10
0.37 sec
================   Topic #:  3
================   Article #:  12
0.50 sec
kmeans_4_부동산_20221201_20221231_crwal_news_context.pkl
================   Topic #:  0
================   Article #:  12
0.25 sec
================   Topic #:  1
================   Article #:  12
0.50 sec
================   Topic #:  2
================   Article #:  12
0.44 sec
================   Topic #

In [27]:
os.listdir('search')

['hdbscan',
 'bertopic(mpnet)',
 'kmeans_12',
 'bertopic(paraphrase)',
 'kmeans_4',
 'bertopic(SR-BERT)',
 'kmeans_8']

In [68]:
line = '삼성전자가반도체생산현장에서일하는설비기술직군엔지니어를대상으로금요일부터일요일까지만일하는주말전담제도입을추진하며이는국내주요대기업중첫사례로삼성전자DS반도체부문은최근설비직군임직원을대상으로한공지에서주말전담제를시범운영한다고안내했다.'
spell_checker.check(f'{line}').checked

'삼성전자가반도체생산현장에서일하는설비기술직군엔지니어를대상으로금요일부터일요일까지만일하는주말전담제도입을추진하며이는국내주요대기업중첫사례로삼성전자DS반도체부문은최근설비직군임직원을대상으로한공지에서주말전담제를시범운영한다고안내했다.'

In [133]:
from pykospacing import Spacing
spacing = Spacing()
spacing.set_rules_by_csv('./text.csv', '단어')

In [70]:
n = spacing(line)

1/1 [==============================] - 0s 31ms/step


In [109]:
text = pd.read_csv('text.csv', index_col=0)
text

,단어
인덱스,
1,마르지엘라 에디션
2,맞았다
3,Crawling
4,에디션


In [112]:

# text = text.append({'인덱스': 4, '단어': '에디션'}, ignore_index = True)
text.to_csv('text.csv')

In [111]:
text.iloc[0] = '에디션이'
text

,단어
인덱스,
1,에디션이
2,맞았다
3,Crawling
4,에디션


In [118]:
line = '18일 한국거래소에 따르면 올해 1월 초부터 이달 16일까지 유가 증권시장과 코스닥시장에서 개인투자자들의 순 매수 상위 10개 종목이 전부 마이너스 수익률을 낸 것으로 집계되었으며 개인투자자들의 순 매수 상위 10개 종목 중 삼성전자, 네이버, 카카오, 삼성전자 우, SK하이닉스, 두산 에너 빌리티, 카카오뱅크, 에코 프로 비엠, LG전자, 삼성전기  은 전부 마이너스 수익률을 냈다.'
line.replace('  ', ' ')

'18일 한국거래소에 따르면 올해 1월 초부터 이달 16일까지 유가 증권시장과 코스닥시장에서 개인투자자들의 순 매수 상위 10개 종목이 전부 마이너스 수익률을 낸 것으로 집계되었으며 개인투자자들의 순 매수 상위 10개 종목 중 삼성전자, 네이버, 카카오, 삼성전자 우, SK하이닉스, 두산 에너 빌리티, 카카오뱅크, 에코 프로 비엠, LG전자, 삼성전기 은 전부 마이너스 수익률을 냈다.'

In [124]:
line = '삼성전자가소비자비욘드비스포크출시4년차를맞아빛이라는컬러와시간이라는소재로완성한비스포크냉장고인피니트라인은남들과다른,나만을위한고유의희소성과시간이지나도변하지않는가치를선사한다.'
line = line.replace(' ', '')
print(line)
line = spell_checker.check(line).checked
print(line)
line = spacing(line)
print(line)
line = line.strip()

삼성전자가소비자비욘드비스포크출시4년차를맞아빛이라는컬러와시간이라는소재로완성한비스포크냉장고인피니트라인은남들과다른,나만을위한고유의희소성과시간이지나도변하지않는가치를선사한다.
삼성전자가소비자비욘드비스포크출시4년차를맞아빛이라는컬러와시간이라는소재로완성한비스포크냉장고인피니트라인은남들과다른,나만을위한고유의희소성과시간이지나도변하지않는가치를선사한다.
1/1 [==============================] - 0s 30ms/step
삼성전자가 소비자 비욘드비스포크 출시 4년차를 맞아 빛이라는 컬러와 시간이라는 소재로 완성한 비스포크 냉장고인피니트 라인은 남들과 다른, 나만을 위한 고유의 희소성과 시간이 지나도 변하지 않는 가치를 선사한다.


삼성전자가 소비자 비욘드 비스포크 출시 4년 차를 맞아 빛이라는 컬러와 시간이라는 소재로 완성한 비스포크 냉장고 인피니트 라인은 남들과 다른, 나만을 위한 고유의 희소성과 시간이 지나도 변하지 않는 가치를 선사한다.

In [130]:
line = '18일 한국거래소에 따르면 올해 1월 초부터 이달 16일까지 유가 증권시장과 코스닥시장에서 개인투자자들의 순 매수 상위 10개 종목이 전부 마이너스 수익률을 낸 것으로 집계되었으며 개인투자자들의 순 매수 상위 10개 종목 중 삼성전자, 네이버, 카카오, 삼성전자 우, SK하이닉스, 두산 에너 빌리티, 카카오뱅크, 에코 프로 비엠, LG전자, 삼성전기  은 전부 마이너스 수익률을 냈다.'
print(line)
line = line.replace('  ', ' ')
print(line)
line = spell_checker.check(line).checked
print(line)
line = spacing(line)
line = re.sub('[\s]{2,}', ' ', line)
print(line)
line = line.strip()

18일 한국거래소에 따르면 올해 1월 초부터 이달 16일까지 유가 증권시장과 코스닥시장에서 개인투자자들의 순 매수 상위 10개 종목이 전부 마이너스 수익률을 낸 것으로 집계되었으며 개인투자자들의 순 매수 상위 10개 종목 중 삼성전자, 네이버, 카카오, 삼성전자 우, SK하이닉스, 두산 에너 빌리티, 카카오뱅크, 에코 프로 비엠, LG전자, 삼성전기  은 전부 마이너스 수익률을 냈다.
18일 한국거래소에 따르면 올해 1월 초부터 이달 16일까지 유가 증권시장과 코스닥시장에서 개인투자자들의 순 매수 상위 10개 종목이 전부 마이너스 수익률을 낸 것으로 집계되었으며 개인투자자들의 순 매수 상위 10개 종목 중 삼성전자, 네이버, 카카오, 삼성전자 우, SK하이닉스, 두산 에너 빌리티, 카카오뱅크, 에코 프로 비엠, LG전자, 삼성전기 은 전부 마이너스 수익률을 냈다.
18일 한국거래소에 따르면 올해 1월 초부터 이달 16일까지 유가 증권시장과 코스닥시장에서 개인투자자들의 순 매수 상위 10개 종목이 전부 마이너스 수익률을 낸 것으로 집계되었으며 개인투자자들의 순 매수 상위 10개 종목 중 삼성전자, 네이버, 카카오, 삼성전자 우, SK하이닉스, 두산 에너 빌리티, 카카오뱅크, 에코 프로 비엠, LG전자, 삼성전기 은 전부 마이너스 수익률을 냈다.
1/1 [==============================] - 0s 34ms/step
18일 한국거래소에 따르면 올해 1월 초부터 이달 16일까지 유가 증권시장과 코스닥시장에서 개인투자자들의 순 매수 상위 10개 종목이 전부 마이너스 수익률을 낸 것으로 집계되었으며 개인투자자들의 순 매수 상위 10개 종목 중 삼성전자, 네이버, 카카오, 삼성전자 우, SK하이닉스, 두산 에너 빌리티, 카카오뱅크, 에코 프로 비엠, LG전자, 삼성전기 은 전부 마이너스 수익률을 냈다.


In [139]:
kkk = pd.read_pickle('search/bertopic(mpnet)/bertopic_윤석열_20221201_20221203_crwal_news_context.pkl')

In [142]:
kkk

,title,description,url,date,context,topic,top_n_words
0,화물연대 운송거부로 11월 무역 70억 달러 적자,대통령 SNS서 복귀 촉구 수출로 글로벌 복합위기 돌파해야 수출 전략 챙겨나갈...,http://www.fnnews.com/news/202212012342298258,20221201,[대통령 SNS 서 복귀 촉구 수출로 글로벌 복합위기 돌파해야 수출 전략 챙겨 나갈...,-1,대통령 - 위기 - 화물 - 윤석열 - 연대 - 경제 - 극복 - 정부 - 중단 - 업무
1,", 화물연대에 호소 업무 중단 끝내고 경제위기 극복에 힘 모아주길 바래",윤석열 대통령이 지난달 29일 서울 용산 대통령실 청사에서 열린 국무회의를 주재하고...,http://www.segye.com/content/html/2022/12/02/2...,20221202,[윤석열 대통령이 지난달 29일 서울 용산 대통령실 청사에서 열린 국무회의를 주재하...,-1,대통령 - 위기 - 화물 - 윤석열 - 연대 - 경제 - 극복 - 정부 - 중단 - 업무
2,"대통령 화물연대, 업무중단 끝내고 경제위기 극복 힘 모아달라 호소",윤석열 대통령. 사진 윤석열 대통령이 최근 집단 운송거부 중인 화물연대를 향해 ...,http://www.fnnews.com/news/202212020744086617,20221202,[윤석열 대통령이 최근 집단 운송 거부 중인 화물연대를 향해 업무 중단을 끝내고 경...,-1,대통령 - 위기 - 화물 - 윤석열 - 연대 - 경제 - 극복 - 정부 - 중단 - 업무
3,재벌 주도 성장정책으론 잃어버린 30년 못 피한다,윤석열 정부 옛날식 재벌 주력산업 의존적 산업정책 회귀 조짐 걱정 윤석열 정부의 ...,https://hankookilbo.com/News/Read/A20221201141...,20221201,[박상인 서울대 행정대학원 교수 윤석열 정부 옛날식 재벌 주력산업 의존적 산업 정책...,-1,대통령 - 위기 - 화물 - 윤석열 - 연대 - 경제 - 극복 - 정부 - 중단 - 업무
4,테슬라는 우리가 품는다 지자체 유치경쟁 불붙었다,국내 지방자치단체들이 테슬라의 생산기지인 기가팩토리 유치전에 본격적으로 뛰어들었...,http://www.mk.co.kr/article/10553048,20221201,[국내 기가 팩토리 건설 기 대속 포항 고양 강원도 유치 나서 세금 감면 규제 완화...,-1,대통령 - 위기 - 화물 - 윤석열 - 연대 - 경제 - 극복 - 정부 - 중단 - 업무
5,"정부, 3년간 방위산업에 1조 이상 금융지원",정부가 내년부터 3년간 방위산업에 1조원 이상 금융지원을 제공한다고 밝혔다. 윤석열...,https://www.khan.co.kr/politics/defense-diplom...,20221201,[1200억원 규모로 방산기술 혁신펀드 조성 중소기업 협력사에 3년간 약 2000억...,-1,대통령 - 위기 - 화물 - 윤석열 - 연대 - 경제 - 극복 - 정부 - 중단 - 업무
6,도이치 주가조작 선수 매도 지시하자 7초 뒤 김건희 계좌 실행,"12시에 3300에 8만개 때려달라 해줘 김아무개 , 준비시킬게요 민아무개 , ...",http://www.hani.co.kr/arti/society/society_gen...,20221202,[구속된 김건희 파일 작성 의심자 재판정에 김건희 권오수 김아무개 주고받기 신문에 ...,-1,대통령 - 위기 - 화물 - 윤석열 - 연대 - 경제 - 극복 - 정부 - 중단 - 업무
7,"정부, IRA 청정에너지 분야 세액공제 2차 의견서 제출",윤석열 대통령과 조 바이든 미국 대통령이 13일 현지시간 캄보디아 프놈펜의 한 호...,http://www.fnnews.com/news/202212021619511946,20221202,[정부는 미국 인플레이션 감축법 IRA 내 청정에너지 분야 세액공제에 대한 2차 정...,-1,대통령 - 위기 - 화물 - 윤석열 - 연대 - 경제 - 극복 - 정부 - 중단 - 업무
8,매출 1조 기업이 가업 시대 역행에 자가당착,한겨레S 뉴스레터 무료 구독. 검색창에 에스레터 를 쳐보세요. 윤석열 정부 들어 ...,http://www.hani.co.kr/arti/economy/economy_gen...,20221203,[김영배의 경제 들여다보기 가업상속공제 윤석열 정부 들어 첫 세제 개편 방안인 올해...,-1,대통령 - 위기 - 화물 - 윤석열 - 연대 - 경제 - 극복 - 정부 - 중단 - 업무
9,K 방산에 3년간 1조 이상 금융지원 수출 탄력,정부가 내년부터 3년간 방위산업에 1조원 이상의 금융지원을 제공한다. 지난달 24일...,http://www.dt.co.kr/contents.html?article_no=2...,20221201,"[윤석열 대통령이 24일 경남 창원시 한화에어로스 페이스를 방문, 전시된 장비를 살...",-1,대통령 - 위기 - 화물 - 윤석열 - 연대 - 경제 - 극복 - 정부 - 중단 - 업무


In [ ]:
folders = [folder for folder in os.listdir('search')]
for folder in folders:
    files = [file for file in os.listdir(f'search/{folder}') if file[-4:] == '.pkl']
    for file in files:
        print('=================')
        print(f'Model: {folder}')
        print(f'File name: {file}')
        summary = pd.read_pickle(f'summary/{folder}/{file}')
        num = 0
        for idx, line in summary.items():
            print('=================생성된 row 문장=================')
            print(idx, line)
            print()
            print('=================hanspell 맞춤법 검사 처리=================')
            line = spell_checker.check(line).checked
            print(idx, line)
            print()
            print('=================띄어쓰기 제거 후 hanspell=================')
            line = line.replace(' ', '')
            line = spell_checker.check(line).checked
            print(idx, line)
            print()
            print('=================띄어쓰기 제거 후 pykospacing=================')
            line = line.replace(' ', '')
            line = spacing(line)
            print(idx, line)
            print()
            print('=================띄어쓰기 제거 후 hanspell + pykospacing=================')
            line = line.replace(' ', '')
            line = spell_checker.check(line).checked
            line = spacing(line)
            line = re.sub('[\s]{2,}', ' ', line)
            print(idx, line)
            print()
            
            # line = line.replace('  ', ' ')
            # line = spell_checker.check(line).checked
            # line = spacing(line)
            # line = line.strip()
            # print(idx, line)
            num += 0
            print()
        # print(summary)
        print('=================')

Model: hdbscan
File name: hdbscan_삼성전자_20221201_20221231_crwal_news_context.pkl
=================생성된 row 문장=================
0 삼성전자가 반도체 생산 현장에서 일하는 설비 기술 직군 엔지니어를 대상으로 금요일부터 일요일까지만 일하는 주말 전담제 도입을 추진하며 이는 국내 주요 대기업 중 첫 사례로 삼성전자 DS 반도체 부문은 최근 설비 직군 임직원을 대상으로 한 공지에서 주말 전담제를 시범 운영한다고 안내했다.

=================hanspell 맞춤법 검사 처리=================
0 삼성전자가 반도체 생산 현장에서 일하는 설비 기술 직군 엔지니어를 대상으로 금요일부터 일요일까지만 일하는 주말 전담제 도입을 추진하며 이는 국내 주요 대기업 중 첫 사례로 삼성전자 DS 반도체 부문은 최근 설비 직군 임직원을 대상으로 한 공지에서 주말 전담제를 시범 운영한다고 안내했다.

=================띄어쓰기 제거 후 hanspell=================
0 삼성전자가반도체생산현장에서일하는설비기술직군엔지니어를대상으로금요일부터일요일까지만일하는주말전담제도입을추진하며이는국내주요대기업중첫사례로삼성전자DS반도체부문은최근설비직군임직원을대상으로한공지에서주말전담제를시범운영한다고안내했다.

=================띄어쓰기 제거 후 pykospacing=================
1/1 [==============================] - 0s 30ms/step
0 삼성전자가 반도체 생산 현장에서 일하는 설비 기술직군 엔지니어를 대상으로 금요일부터 일요일까지만 일하는 주말 전담제 도입을 추진하며 이는 국내 주요 대기업 중 첫 사례로 삼성전자 DS반도체 부문은 최근 설비 직군 임직원을 대상으로 한 공지에서 주말 전담제를 시범 운영한다고 안내했다.

=================띄어쓰기 제거 후 hanspell + pykospac

In [ ]:
text_mod = re.sub('',"apple",text) 

In [ ]:
folders = [folder for folder in os.listdir('search')]
for folder in folders:
    files = [file for file in os.listdir(f'search/{folder}') if file[-4:] == '.pkl']
    for file in files:
        print('=================')
        print(f'Model: {folder}')
        print(f'File name: {file}')
        summary = pd.read_pickle(f'summary/{folder}/{file}')
        num = 0
        for idx, line in summary.items():
            line = line.replace(' ', '')
            line = spell_checker.check(f'{line}').checked
            # line = spacing(line)
            print(idx, line)
            num += 0
            print()
        # print(summary)
        print('=================')

In [34]:
pd.read_pickle('search/bertopic(mpnet)/bertopic_부동산_20221201_20221215_crwal_news_context.pkl')

1003

- pykospacing
- hanspell
- chatspace


In [ ]:
from transformers import BertModel, BertTokenizer

model_name_or_path = "LOCAL_MODEL_PATH"  # Bert 바이너리가 포함된 디렉토리

model = BertModel.from_pretrained(model_name_or_path, add_pooling_layer=False)
tokenizer = BertTokenizer.from_pretrained(model_name_or_path)